In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from pandatorch.data import DataFrame

In [3]:
df=pd.read_csv("IRIS.csv")

In [4]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [5]:
torch_df=DataFrame(X=df.drop("species",axis=1),y=df['species'])

In [6]:
torch_df.df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [7]:
torch_df.features[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]], dtype=float32)

In [8]:
torch_df.target[:5]

array([0., 0., 0., 0., 0.], dtype=float32)

In [9]:
torch_df.class_to_idx

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader

In [11]:
loader=DataLoader(torch_df)

In [12]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1=nn.Linear(in_features=torch_df.get_number_of_columns(torch_df.features),out_features=5)
        self.output=nn.Linear(5,3)
        self.softmax=nn.Softmax(dim=1)
        self.relu=nn.ReLU()
    
    def forward(self,x):
        x=self.relu(self.l1(x))
        x=self.softmax(self.output(x))
        return x

In [13]:
model = Net()
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()


In [14]:
def train(model, dataLoader, optimizer, criterion):
    for epoch in range(2):
        running_loss = 0.0
        for i, data in enumerate(dataLoader):
            # data is a list of [inputs, labels]
            inputs, labels = data

            # Zero the param gradients
            optimizer.zero_grad()

            # forward+backward+optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            print(f"Epoch:{epoch+1}\nLoss:{running_loss/2000}")
            running_loss = 0


In [15]:
train(model,loader,optimizer,criterion)

RuntimeError: expected scalar type Long but found Float